In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd
import numpy as np
import csv

import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import time
from time import sleep

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import ssl

import schedule
import time
from datetime import datetime

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
import pyrebase
import datetime
from dateutil.parser import parse
from urllib.request import urlopen
context=ssl._create_unverified_context()

In [4]:
import datetime
import time

## https://www.dreamspon.com/ crawling

In [304]:
#드림스폰의 day를 크롤링해오는 코드임.
def days_dreams():
    dday_bef = []
    page_num = 1
    while(page_num <=3):        
        url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
        req = urllib.request.urlopen(url)
        res = req.read()
        soup = BeautifulSoup(res,'html.parser')
        days = soup.select(" .td_day > .count")        
        for i in range(len(days)):
            if 'D+' in str(days[i].text): # D+는 모집이 마감된 날이므로 크롤링 해와서는 안됨.
                pass
            else:
                
                dday_bef.append((days[i].text).strip("D-"))
        page_num += 1  
    return dday_bef

In [305]:
def link_title_dreams():
    titles = []
    links= []
    page_num = 1
    while(page_num <=3):        
        url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
        req = urllib.request.urlopen(url)
        res = req.read()
        soup = BeautifulSoup(res,'html.parser')
        contests = soup.find_all("p",class_="title")
        days = soup.select(" .td_day > .count")    
        for i in range(len(days)):
            if 'D+' in str(days[i].text):
                pass
            else:
                contests[i]  = ((str(contests[i]).strip('<p class="title"><a href="')).strip('</a></p>'))
                link, title =  contests[i].split('">')
                links.append("https://www.dreamspon.com/" + link)
                titles.append(title)
        page_num += 1  
    return links, titles

In [306]:
# def link_dreams(link_test):
#     link_bef=[]
#     page_num = 1
#     for t in range(len(link_test)):
#         link_address, title_name = link_test[t].split('">')
#         link_ver1 = "https://www.dreamspon.com/" + link_address
#         link_bef.append(link_ver1)
#     page_num += 1  
#     return link_bef

In [307]:
# def titles_dreams(link_test):
#     titles_bef = [] # 행사 이름
#     page_num = 1
#     for t in range(len(link_test)):
#         link_address, title_name = link_test[t].split('">')
#         titles_bef.append(title_name)
#     page_num += 1  
#     return titles_bef

In [308]:
def insts_dreams():  #기관 크롤링 코드
    inst = []
    page_num = 1
    while(page_num <=3):
        url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
        req = urllib.request.urlopen(url)
        res = req.read()
        soup = BeautifulSoup(res,'html.parser')
        idx = 1
        while(idx<=60):
            if 'D-' in str(soup.select("tr>td")[idx+1].text):
                inst.append(soup.select("tr>td")[idx].text) #       기관에 해당되는 ind는 : 1,5,9,13 의 형태다
            idx += 4                                        #       따라서 +4를 해줘야 한다.  
        page_num += 1
    return inst

In [309]:
def s_e_test_dreams(list_adress):
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(5)
    login_path ='//*[@id="loginForm"]/div[1]/input'
    driver.get(list_adress)
    result = driver.switch_to_alert()
    result.accept()
    driver.find_element_by_name('mbr_id').send_keys('rainrain16@hanmail.net')
    driver.find_element_by_name('pwd_in').send_keys('rainrain16')
    driver.find_element_by_xpath(login_path).click()
    sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    crawl_data = str(soup.find_all("li", class_= "day"))
    return crawl_data

In [310]:
def s_e_days_dreams():
    crawl=[]
    links, titles = link_title_dreams()
    for i in range(len(links)):
        crawl.append(s_e_test_dreams(links[i]))
    return crawl

In [311]:
def s_e_preprocess():
    s_e_pre = s_e_days_dreams()
    
    for i in range(len(s_e_pre)):
        s_e_pre[i] = (s_e_pre[i].strip('[<li class="day" style="height:auto;">\n<p class="">')).strip('</p> </li>]')

    return s_e_pre

In [312]:
#날짜를 가지고 오는 코드인데, 상당히 까다로움. 크롬을 하나하나 켜야한다는 단점이 있음
def s_e_final():
    s_e_pre = s_e_preprocess()

    start_bef = []
    end_bef = []
    for i in range(len(s_e_pre)):
        if '(1차)' not in s_e_pre[i]: #(1차)는 사실상 마감된 코드이므로 크롤링해와서는 안된다.
            sen1, sen2 = s_e_pre[i].split('~ ')
            start, noise1 = sen1.split(". (")
            end, noise2 = sen2.split(". (")
            start_bef.append(start)
            end_bef.append(end)

        else:
            if '(3차)' in s_e_pre[i]:
                first, second = s_e_pre[i].split('(2차) ')
                second, third = second.split("</p>")
                start, end = second.split('~ ')
                start = start.replace('(1차) ', "")
                start_date, noise1 = start.split(". (")
                end_date, noise2 = end.split(". (")


                start_bef.append(start_date)
                end_bef.append(end_date)
            else:
                first, second = s_e_pre[i].split('(2차) ')
                start, end = second.split('~ ')
                start = start.replace('(1차) ', "")
                start_date, noise1 = start.split(". (")
                end_date, noise2 = end.split(". (")


                start_bef.append(start_date)
                end_bef.append(end_date)

    return start_bef, end_bef
# 날짜가 2021. 01. 02. ~ 2021. 02. 21. 이런식으로 들어가 있기 때문에 split과 strip을 적절해 사용해야 함.

In [313]:
def final_dreams():
    dday =  days_dreams()
    links,title = link_title_dreams()
    start, end =  s_e_final()
    inst = insts_dreams()
    print(len(dday), len(links), len(title), len(start), len(end), len(inst))
    
    tabl_data_bef = {'type':'3 장학금', 'qualification': "대학생",'title': title, 'notice': start,  'deadline': end, 'dday':dday, 'sponsor':inst, 'title2': title,'link':links}
    df_bef = pd.DataFrame(tabl_data_bef, columns=['type', 'qualification', 'title', 'notice', 'deadline', 'dday', 'sponsor', 'title2', 'link'])
    return df_bef

In [314]:
df_dreams = final_dreams()

<ipython-input-309-7dd2de8f4e2f>:6: DeprecationWarning: use driver.switch_to.alert instead
  result = driver.switch_to_alert()


45 45 45 45 45 45


In [315]:
df_dreams

,type,qualification,title,notice,deadline,dday,sponsor,title2,link
0,3 장학금,대학생,김포시 독서동아리 지원,2021. 07. 02,2021. 07. 11,0,김포시,김포시 독서동아리 지원,https://www.dreamspon.com/scholarship/view.htm...
1,3 장학금,대학생,BNK금융그룹 희망나눔장학 (2021년),2021. 07. 01,2021. 07. 12,1,BNK금융그룹희망나눔재단,BNK금융그룹 희망나눔장학 (2021년),https://www.dreamspon.com/scholarship/view.htm...
2,3 장학금,대학생,시흥시 청년문화 활성화 지원,2021. 06. 25,2021. 07. 12,1,시흥시,시흥시 청년문화 활성화 지원,https://www.dreamspon.com/scholarship/view.htm...
3,3 장학금,대학생,안양시 인방동물센터 지정장학,2021. 06. 16,2021. 07. 13,2,안양시인재육성재단,안양시 인방동물센터 지정장학,https://www.dreamspon.com/scholarship/view.htm...
4,3 장학금,대학생,한국전력전우회 섬사랑 장학 (2021년),2021. 06. 14,2021. 07. 13,2,한국전력전우회,한국전력전우회 섬사랑 장학 (2021년),https://www.dreamspon.com/scholarship/view.htm...
5,3 장학금,대학생,영광군 청년취업활동 지원,2021. 07. 01,2021. 07. 14,3,영광군,영광군 청년취업활동 지원,https://www.dreamspon.com/scholarship/view.htm...
6,3 장학금,대학생,경기 여성청소년 생리용품 지원,2021. 07. 01,2021. 07. 14,3,경기도,경기 여성청소년 생리용품 지원,https://www.dreamspon.com/scholarship/view.htm...
7,3 장학금,대학생,성련장학재단 장학 (2021년 2학기),2021. 06. 28,2021. 07. 14,3,성련장학재단,성련장학재단 장학 (2021년 2학기),https://www.dreamspon.com/scholarship/view.htm...
8,3 장학금,대학생,덕신하우징 골프 장학,2021. 06. 15,2021. 07. 14,3,덕신하우징,덕신하우징 골프 장학,https://www.dreamspon.com/scholarship/view.htm...
9,3 장학금,대학생,해양수산부장학 (수산후계 장학) (2021년),2021. 06. 21,2021. 07. 15,4,농어촌희망재단,해양수산부장학 (수산후계 장학) (2021년),https://www.dreamspon.com/scholarship/view.htm...


## 1. ddays test

In [26]:
#드림스폰의 day를 크롤링해오는 코드임.

dday_bef = []
page_num = 1
while(page_num <=3):        
    url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
    req = urllib.request.urlopen(url)
    res = req.read()
    soup = BeautifulSoup(res,'html.parser')
    days = soup.select(" .td_day > .count")        
    for i in range(len(days)):
        if 'D+' in str(days[i].text): # D+는 모집이 마감된 날이므로 크롤링 해와서는 안됨.
            pass
        else:

            dday_bef.append((days[i].text).strip("D-"))
    page_num += 1  


In [27]:
print(len(dday_bef))

45


## 2. link test dreams test

In [84]:
titles = []
links= []
page_num = 1
while(page_num <=3):        
    url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
    req = urllib.request.urlopen(url)
    res = req.read()
    soup = BeautifulSoup(res,'html.parser')
    contests = soup.find_all("p",class_="title")
    days = soup.select(" .td_day > .count")    
    for i in range(len(days)):
        if 'D+' in str(days[i].text):
            pass
        else:
            contests[i]  = ((str(contests[i]).strip('<p class="title"><a href="')).strip('</a></p>'))
            link, title =  contests[i].split('">')
            links.append("https://www.dreamspon.com/" + link)
            titles.append(title)
    page_num += 1  

In [91]:
len(links)

45

In [71]:
links            

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,


## 3. start, end day test

In [102]:
def s_e_test_dreams(list_adress):
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(10)
    login_path ='//*[@id="loginForm"]/div[1]/input'
    driver.get(list_adress)
    result = driver.switch_to_alert()
    result.accept()
    driver.find_element_by_name('mbr_id').send_keys('rainrain16@hanmail.net')
    driver.find_element_by_name('pwd_in').send_keys('rainrain16')
    driver.find_element_by_xpath(login_path).click()
    sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    crawl_data = str(soup.find_all("li", class_= "day"))
    return crawl_data

In [250]:
# crawl=[]
# links, titles = link_title_dreams()
# for i in range(len(links)):
#     crawl.append(s_e_test_dreams(links[i]))
    
    
def s_e_days_dreams():
    crawl=[]
    links, titles = link_title_dreams()
    for i in range(len(links)):
        crawl.append(s_e_test_dreams(links[i]))
    return crawl

In [251]:
crawl = s_e_days_dreams()

<ipython-input-234-7dd2de8f4e2f>:6: DeprecationWarning: use driver.switch_to.alert instead
  result = driver.switch_to_alert()


In [252]:
crawl

['[<li class="day" style="height:auto;">\n<p class="">2021. 07. 02. (금) ~ 2021. 07. 11. (일)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 12. (월)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 25. (금) ~ 2021. 07. 12. (월)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 16. (수) ~ 2021. 07. 13. (화)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 14. (월) ~ 2021. 07. 13. (화)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 28. (월) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 15. (화) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2

In [253]:
crawl2 = crawl.copy()

In [254]:
crawl2

['[<li class="day" style="height:auto;">\n<p class="">2021. 07. 02. (금) ~ 2021. 07. 11. (일)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 12. (월)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 25. (금) ~ 2021. 07. 12. (월)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 16. (수) ~ 2021. 07. 13. (화)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 14. (월) ~ 2021. 07. 13. (화)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 07. 01. (목) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 28. (월) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2021. 06. 15. (화) ~ 2021. 07. 14. (수)</p> </li>]',
 '[<li class="day" style="height:auto;">\n<p class="">2

In [255]:

s_e_pre = crawl2

for i in range(len(s_e_pre)):
    s_e_pre[i] = (s_e_pre[i].strip('[<li class="day" style="height:auto;">\n<p class="end">')).strip('</p> </li>]')


In [256]:
s_e_pre

['2021. 07. 02. (금) ~ 2021. 07. 11. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 12. (월)',
 '2021. 06. 25. (금) ~ 2021. 07. 12. (월)',
 '2021. 06. 16. (수) ~ 2021. 07. 13. (화)',
 '2021. 06. 14. (월) ~ 2021. 07. 13. (화)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 06. 28. (월) ~ 2021. 07. 14. (수)',
 '2021. 06. 15. (화) ~ 2021. 07. 14. (수)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 28. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 04. 19. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 29. (화) ~ 2021. 07. 18. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 20. (화)',
 '2021. 07. 05. (월) ~ 2021. 07. 22. (목)',
 '2021. 07. 05. (월) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 06. 23. (수) ~ 2021. 07. 23. (금)',
 '2021. 07. 19. (월) ~ 2021. 07. 23. (금)',
 '2021. 02. 08. (월) ~ 2021. 07. 23

In [296]:
test = '(1차) 2021. 06. 01. (화) ~ 2021. 06. 30. (수)</p><p class="">(2차) 2021. 07. 01. (목) ~ 2021. 07. 31. (토)</p><p class="">(3차) 2021. 08. 01. (목) ~ 2021. 08. 31. (화)'


first, second =test.split('(2차) ')
second.split("</p>")
# second, third = second.split("</p>")

# start, end = second.split('~ ')
# start = start.replace('(1차) ', "")
# start_date, noise1 = start.split(". (")
# start_date

# end
# end_date, noise1 = end.split(". (")
# start_date

['2021. 07. 01. (목) ~ 2021. 07. 31. (토)',
 '<p class="">(3차) 2021. 08. 01. (목) ~ 2021. 08. 31. (화)']

In [190]:
start.replace('(1차) ', "")

'2021. 07. 15. (목) '

In [172]:
start_bef = []
end_bef = []
for i in range(len(s_e_pre)):
    if '(1차)' not in s_e_pre[i]: #(1차)는 사실상 마감된 코드이므로 크롤링해와서는 안된다.

        start_day, end_day_ver1  = (s_e_pre[i]).split('~')
        end_day_ver2, end_day_ver3 =  (end_day_ver1).split("<span>D")
        start_bef.append((start_day.strip('.')).strip('. '))
        end_bef.append(end_day_ver2.strip('.'))

    elif:            

        start_ver1, start_ver2 = (s_e_pre[i]).split('</span><br/>')
        start_ver3, end_ver1 = start_ver1.split('~')
        end_ver2, end_ver3 = end_ver1.split('<span')

        start_ver4 ,end_ver4  = start_ver2.split('~')

        end_ver5, end_ver6 = end_ver4.split('<span')


        start_bef.append('2' + start_ver4.strip("(2차) ").strip(". ").strip("."))
        end_bef.append(end_ver5.strip(".").strip(" "))




ValueError: not enough values to unpack (expected 2, got 1)

In [130]:
test = '2021. 07. 02. (금) ~ 2021. 07. 11. (일)'
sen1, sen2 = test.split('~ ')
start, noise1 = sen1.split(". (")
end, noise2 = sen2.split(". (")
start

'2021. 07. 02'

In [215]:
crawl

['2021. 07. 02. (금) ~ 2021. 07. 11. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 12. (월)',
 '2021. 06. 25. (금) ~ 2021. 07. 12. (월)',
 '2021. 06. 16. (수) ~ 2021. 07. 13. (화)',
 '2021. 06. 14. (월) ~ 2021. 07. 13. (화)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 06. 28. (월) ~ 2021. 07. 14. (수)',
 '2021. 06. 15. (화) ~ 2021. 07. 14. (수)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 28. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 04. 19. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 29. (화) ~ 2021. 07. 18. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 20. (화)',
 '2021. 07. 05. (월) ~ 2021. 07. 22. (목)',
 '2021. 07. 05. (월) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 06. 23. (수) ~ 2021. 07. 23. (금)',
 '2021. 07. 19. (월) ~ 2021. 07. 23. (금)',
 '2021. 02. 08. (월) ~ 2021. 07. 23

In [141]:
def s_e_preprocess():
    start_list = []
    end_list = []
    s_e_pre = s_e_days_dreams()
    
    for i in range(len(s_e_pre)):
        s_e_pre[i] = (s_e_pre[i].strip('[<li class="day" style="height:auto;">\n<p class="">')).strip('</p> </li>]')
    return s_e_pre

In [217]:

start_list = []
end_list = []
s_e_pre = crawl

for i in range(len(s_e_pre)):
    s_e_pre[i] = (s_e_pre[i].strip('[<li class="day" style="height:auto;">\n<p class="">')).strip('</p> </li>]')

In [297]:
s_e_pre

['2021. 07. 02. (금) ~ 2021. 07. 11. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 12. (월)',
 '2021. 06. 25. (금) ~ 2021. 07. 12. (월)',
 '2021. 06. 16. (수) ~ 2021. 07. 13. (화)',
 '2021. 06. 14. (월) ~ 2021. 07. 13. (화)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 07. 01. (목) ~ 2021. 07. 14. (수)',
 '2021. 06. 28. (월) ~ 2021. 07. 14. (수)',
 '2021. 06. 15. (화) ~ 2021. 07. 14. (수)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 15. (목)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 28. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 21. (월) ~ 2021. 07. 16. (금)',
 '2021. 04. 19. (월) ~ 2021. 07. 16. (금)',
 '2021. 06. 29. (화) ~ 2021. 07. 18. (일)',
 '2021. 07. 01. (목) ~ 2021. 07. 20. (화)',
 '2021. 07. 05. (월) ~ 2021. 07. 22. (목)',
 '2021. 07. 05. (월) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 07. 01. (목) ~ 2021. 07. 23. (금)',
 '2021. 06. 23. (수) ~ 2021. 07. 23. (금)',
 '2021. 07. 19. (월) ~ 2021. 07. 23. (금)',
 '2021. 02. 08. (월) ~ 2021. 07. 23

In [298]:
#날짜를 가지고 오는 코드인데, 상당히 까다로움. 크롬을 하나하나 켜야한다는 단점이 있음


start_bef = []
end_bef = []
for i in range(len(s_e_pre)):
    if '(1차)' not in s_e_pre[i]: #(1차)는 사실상 마감된 코드이므로 크롤링해와서는 안된다.
        sen1, sen2 = s_e_pre[i].split('~ ')
        start, noise1 = sen1.split(". (")
        end, noise2 = sen2.split(". (")
        start_bef.append(start)
        end_bef.append(end)

    else:
        if '(3차)' in s_e_pre[i]:
            first, second = s_e_pre[i].split('(2차) ')
            second, third = second.split("</p>")
            start, end = second.split('~ ')
            start = start.replace('(1차) ', "")
            start_date, noise1 = start.split(". (")
            end_date, noise2 = end.split(". (")


            start_bef.append(start_date)
            end_bef.append(end_date)
        else:
            first, second = s_e_pre[i].split('(2차) ')
            start, end = second.split('~ ')
            start = start.replace('(1차) ', "")
            start_date, noise1 = start.split(". (")
            end_date, noise2 = end.split(". (")


            start_bef.append(start_date)
            end_bef.append(end_date)
            


# 날짜가 2021. 01. 02. ~ 2021. 02. 21. 이런식으로 들어가 있기 때문에 split과 strip을 적절해 사용해야 함.

In [302]:
 start_bef

['2021. 07. 02',
 '2021. 07. 01',
 '2021. 06. 25',
 '2021. 06. 16',
 '2021. 06. 14',
 '2021. 07. 01',
 '2021. 07. 01',
 '2021. 06. 28',
 '2021. 06. 15',
 '2021. 06. 21',
 '2021. 06. 21',
 '2021. 06. 21',
 '2021. 06. 28',
 '2021. 06. 21',
 '2021. 04. 19',
 '2021. 06. 29',
 '2021. 07. 01',
 '2021. 07. 05',
 '2021. 07. 05',
 '2021. 07. 01',
 '2021. 07. 01',
 '2021. 06. 23',
 '2021. 07. 19',
 '2021. 02. 08',
 '2021. 06. 07',
 '2021. 07. 20',
 '2021. 07. 19',
 '2021. 08. 02',
 '2021. 07. 01',
 '2021. 07. 01',
 '2021. 06. 14',
 '2021. 07. 01',
 '2021. 07. 01',
 '2021. 06. 05',
 '2021. 06. 14',
 '2021. 05. 24',
 '2021. 06. 28',
 '2021. 07. 05',
 '2021. 07. 01',
 '2021. 07. 13',
 '2021. 07. 01',
 '2021. 07. 01',
 '2021. 10. 25',
 '2021. 08. 30',
 '2021. 05. 02']

In [ ]:
#날짜를 가지고 오는 코드인데, 상당히 까다로움. 크롬을 하나하나 켜야한다는 단점이 있음
def s_e_final():
    s_e_pre = s_e_preprocess()
    start_bef = []
    end_bef = []
    for i in range(len(s_e_pre)):
        if '(1차)' not in s_e_pre[i]: #(1차)는 사실상 마감된 코드이므로 크롤링해와서는 안된다.
            sen1, sen2 = s_e_pre[i].split('~ ')
            start, noise1 = sen1.split(". (")
            end, noise2 = sen2.split(". (")
            start_bef.append(start)
            end_bef.append(end)

        else:            
            first, second = test.split('(2차)')
            start, end = second.split('~ ')
            start = start.replace('(1차) ', "")
            start_date, noise1 = start.split(". (")
            end_date, noise2 = end.split(". (")
            start_bef.append(start_date)
            end_bef.append(end_date)

            
            start_bef.append(start_bef)
            end_bef.append(end_bef)

    return start_bef, end_bef

# 날짜가 2021. 01. 02. ~ 2021. 02. 21. 이런식으로 들어가 있기 때문에 split과 strip을 적절해 사용해야 함.

## 4. Inst test

In [142]:

inst = []
page_num = 1
while(page_num <=1):
    url = 'https://www.dreamspon.com/scholarship/list.html?page=' + str(page_num)
    req = urllib.request.urlopen(url)
    res = req.read()
    soup = BeautifulSoup(res,'html.parser')
    idx = 1
    while(idx<=60):
        if 'D-' in str(soup.select("tr>td")[idx+1].text):
            inst.append(soup.select("tr>td")[idx].text) #       기관에 해당되는 ind는 : 1,5,9,13 의 형태다
        idx += 4                                        #       따라서 +4를 해줘야 한다.  
    page_num += 1


In [143]:
inst

['김포시',
 'BNK금융그룹희망나눔재단',
 '시흥시',
 '안양시인재육성재단',
 '한국전력전우회',
 '영광군',
 '경기도',
 '성련장학재단',
 '덕신하우징',
 '농어촌희망재단',
 '농어촌희망재단',
 '대한불교진흥원',
 '경기도',
 '한국증권금융 꿈나눔재단',
 '신용회복위원회']